# Check the manually constructed dataset of mining areas and ground truth masks

In [ ]:
import geopandas as gpd
import leafmap
from shapely.ops import unary_union
from shapely.geometry import Point, mapping, box, shape
import shapely
import json
import os
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("..")

os.getcwd()
os.chdir("..")
root = os.path.dirname(os.getcwd())
# root = root + "/workspaces/mine-segmentation" # uncomment when running in Lightning Studios
root

In [ ]:
raw_dataset = root + "/data/raw/mining_tiles_with_masks.gpkg"
post_dataset = root + "/data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg"

# RAW DATASET

In [ ]:
gpd.list_layers(raw_dataset)

In [ ]:
# Load the data
data = gpd.read_file(raw_dataset, layer="tiles")
print(data.shape)
data.head()

In [ ]:
# check distribution in minetype1 column
data["minetype1"].value_counts()

In [ ]:
# check distribution in source_dataset column
data.source_dataset.value_counts()

In [ ]:
# filter out only tiles with valid surface mines
data = data[data["minetype1"].isin(["Surface", "Placer"]) & data["source_dataset"].isin(["maus", "tang", "both"])]
print(data.shape)
data.source_dataset.value_counts()

## Overlap

In [ ]:
# check how many tiles are overlapping with other tiles
data["overlaps"] = data["geometry"].apply(lambda x: data["geometry"].apply(lambda y: x.overlaps(y)).sum())
data["overlaps"].value_counts()

In [ ]:
# calculate overlap percentage of each tile
data["overlap_percentage"] = data["geometry"].apply(lambda x: data["geometry"].apply(lambda y: x.intersection(y).area/x.area).sum() - 1)
data["overlap_percentage"].hist(bins=50)

In [ ]:
# assign validation label to the tiles with overlap percentage less than 0.1
data["validation_eligible"] = data["overlap_percentage"].apply(lambda x: 1 if x < 0.1 else 0)
data["validation_eligible"].value_counts()

In [ ]:
# Create a Leaflet map
m = leafmap.Map()

style_eligible = {
    "stroke": True,
    "color": "green",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "green",
}

# change the color of the tiles with overlap percentage less than 0.1
m.add_gdf(data[data["validation_eligible"] == 1], layer_name="validation_eligible", style=style_eligible)

style_not_eligible = {
    "stroke": True,
    "color": "red",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "red",
}

# change the color of the tiles with overlap percentage more than 0.1
m.add_gdf(data[data["validation_eligible"] == 0], layer_name="validation_not_eligible", style=style_not_eligible)

# Display the map
m

### Group Overlapping Tiles


In [ ]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Polygon

# Assuming `data` is a GeoDataFrame with a 'geometry' column containing Polygon geometries

# Step 1: Create a graph
G = nx.Graph()

# Step 2: Add nodes
for idx, geom in data.iterrows():
    G.add_node(idx, geometry=geom.geometry)

# Step 3: Add edges for overlapping or touching geometries
for i, geom1 in data.iterrows():
    for j, geom2 in data.iterrows():
        if i != j and (geom1.geometry.overlaps(geom2.geometry) or geom1.geometry.touches(geom2.geometry)):
            G.add_edge(i, j)

# Step 4: Find connected components
connected_components = list(nx.connected_components(G))

# Step 5: Assign group IDs
group_id = 0
data['overlap_group'] = -1
for component in connected_components:
    for idx in component:
        data.at[idx, 'overlap_group'] = group_id
    group_id += 1

print(f"Number of connected components: {len(connected_components)}")

data.head()

In [ ]:
# Create a Leaflet map
m = leafmap.Map()
m.add_gdf(data)
m

### Plot groups by color

In [ ]:
import geopandas as gpd
import leafmap
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Step 2: Define styles
def get_group_color(group_id, num_groups):
    cmap = plt.get_cmap('tab20')  # Use a colormap with enough distinct colors
    norm = mcolors.Normalize(vmin=0, vmax=num_groups - 1)
    return mcolors.to_hex(cmap(norm(group_id)))

# Get the number of unique groups
num_groups = data['overlap_group'].nunique()

# Step 3: Add tiles to the map
m = leafmap.Map()

for group_id in data['overlap_group'].unique():
    group_data = data[data['overlap_group'] == group_id]
    color = get_group_color(group_id, num_groups)
    style = {
        "stroke": True,
        "color": color,
        "weight": 2,
        "opacity": 1,
        "fill": True,
        "fillColor": color,
        "fillOpacity": 0.5,
    }
    m.add_gdf(group_data, layer_name=f"group_{group_id}", style=style)

# Display the map
m

## Global distribution of tiles

In [ ]:
# Using Markers on leafmap
def plot_tiles_on_basemap(gdf):
    import folium

    m = folium.Map()

    # Iterate over the tiles and add markers to the map
    for _, row in gdf.iterrows():
        lat = row.geometry.centroid.y
        lon = row.geometry.centroid.x
        folium.Marker([lat, lon]).add_to(m)

    # Display the map
    return m

plot_tiles_on_basemap(data)

# PROCESSED DATASET

In [ ]:
# list layernames 
gpd.list_layers(post_dataset)

In [ ]:
# load the processed dataset
tiles = gpd.read_file(post_dataset, layer="tiles")
maus_poly = gpd.read_file(post_dataset, layer="maus_polygons")
tang_poly = gpd.read_file(post_dataset, layer="tang_polygons")
preferred_poly = gpd.read_file(post_dataset, layer="preferred_polygons")
maus_bbox = gpd.read_file(post_dataset, layer="maus_bboxes")
tang_bbox = gpd.read_file(post_dataset, layer="tang_bboxes")
preferred_bbox = gpd.read_file(post_dataset, layer="preferred_bboxes")
tiles

Construct one coherent dataset with the preferred mask

## Visualize the dataset

In [ ]:
ROW = 6

m = leafmap.Map(center = (tiles.iloc[ROW:ROW+1].geometry.centroid.y.values[0], tiles.iloc[ROW:ROW+1].geometry.centroid.x.values[0]), zoom = 12)
# add satellite
m.add_basemap("SATELLITE")

style_tile = {
    "color": "orange",
    "fillColor": "orange",
    "fillOpacity": 0.0,
}

style_bbox = {
    "color": "green",
    "fillColor": "green",
    "fillOpacity": 0.0,
}

m.add_gdf(tiles.iloc[ROW:ROW+1], layer_name="tiles", style=style_tile)
m.add_gdf(preferred_poly.iloc[ROW:ROW+1], layer_name="masks")
m.add_gdf(preferred_bbox.iloc[ROW:ROW+1], layer_name="bboxes", style=style_bbox)
m

## FILTERED DATASET
This includes the train/test/val split

In [ ]:
# only tiles with split
data = tiles[tiles["split"].notnull()]
print(data.shape)
data.head(2)

In [ ]:
# Provide unique value counts for categorical columns
categorical_columns = ['source_dataset', 'preferred_dataset', 'minetype1', 'minetype2', 'split']
for col in categorical_columns:
    print(f"Unique values in {col}:")
    print(data[col].value_counts())

## Visualize

In [ ]:
# Create a Leaflet map
m = leafmap.Map(height="800px")

# add satellite layer
# m.add_basemap("SATELLITE")

style_train = {
    "stroke": True,
    "color": "green",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "green",
}

style_val = {
    "stroke": True,
    "color": "blue",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "blue",
}

style_test = {
    "stroke": True,
    "color": "red",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "red",
}

# Add train tiles to the map
m.add_gdf(data[data["split"] == "train"], layer_name="train", style=style_train)

# Add val tiles to the map
m.add_gdf(data[data["split"] == "val"], layer_name="val", style=style_val)

# Add test tiles to the map
m.add_gdf(data[data["split"] == "test"], layer_name="test", style=style_test)

# Display the map
m

In [ ]:
# Create a Leaflet map
m = leafmap.Map()

# Add the centroids of each test tile as markers
test_tiles = data[data["split"] == "test"]
for _, row in test_tiles.iterrows():
    lat = row.geometry.centroid.y
    lon = row.geometry.centroid.x
    m.add_marker(location=[lat, lon])

m

## Check Train Test Splits

In [ ]:
# train test split for the combined dataset
print(f"The train/val/test split for the combined dataset is: {data['split'].value_counts(normalize=True)}")

# train test split for the maus dataset
print(f"The train/val/test split for the maus dataset is: {data[data['source_dataset'].isin(['maus', 'both'])]['split'].value_counts(normalize=True)}")

# train test split for the tang dataset
print(f"The train/val/test split for the tang dataset is: {data[data['source_dataset'].isin(['tang', 'both'])]['split'].value_counts(normalize=True)}")


In [ ]:
# check the source dataset in the test split
data[data["split"] == "test"]["source_dataset"].value_counts()

In [ ]:
data[data["split"] == "test"]